In [10]:
import requests

resp = requests.post("http://localhost:7071/api/getNodeDetails", json={"node_id": "integrals"})
print(resp.json())

{'topic': ['Integrals'], 'status': ['unstarted'], 'pk': ['pk']}


In [5]:
from enum import Enum
test = {"test":1, "test2":2}
print("\n- ".join(test.keys()))

test
- test2


In [3]:
import ast
from typing import List, Optional, Type
import azure.functions as func
from psycopg2 import pool
from pydantic import BaseModel, Field, ValidationError, root_validator, validator, ConfigDict
from enum import Enum
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_tool
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.output_parsers import OutputFixingParser, PydanticOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from operator import itemgetter
import re
import json
import os
import subprocess
from ansi2html import Ansi2HTMLConverter
from langchain.agents import AgentExecutor

os.environ["OPENAI_API_KEY"] = "10388fa13cbd4ca0aa189687aa4b3af1"
os.environ['AZURE_OPENAI_ENDPOINT'] = "https://gpt-4-beta-canada.openai.azure.com/"

gpt_4_llm = AzureChatOpenAI(deployment_name="gpt-4-turbo", api_version="2023-07-01-preview", model_name="gpt-4-1106-preview", temperature=0, max_retries=10)

class CodeGenerateSchema(BaseModel):
    code: str = Field(description="Manim code that generates the visuals for the scene. This field MUST be pure python code, that could be put in its own file and executed as-is.")
    class_name: str = Field(description="The Manim scene class to use when running the manim -pql scene.py <class_name> command to render the video")

    @validator("code")
    def is_code(cls, v):
        try:
            parsed_module = ast.parse(v)
            class_names = [node.name for node in ast.walk(parsed_module) if isinstance(node, ast.ClassDef)]
            if not any([v['class_name'] in name for name in class_names]):
                raise ValueError("Target render class not found in code!")
        except SyntaxError:
            raise ValueError("Code is not valid Python")
        return v
    
class VideoGenerate(BaseTool):
    name = "RenderVideo"
    description = "Attempts to render the provided code into a video file and provides the stdout and stderr of the render process"
    args_schema: Type[BaseModel] = CodeGenerateSchema
    conv = Ansi2HTMLConverter()

    # model_config = ConfigDict(from_attributes=True)

    def _run(self, code: str, class_name: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """generate a video synchronously."""
        # write the code to a file
        with open("scene.py", "w") as scene:
            scene.write(code)

        stdout, stderr = subprocess.Popen(f"manim -pql scene.py {class_name}", stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=True).communicate()
        stdout_decoded = stdout.decode('utf-8')
        stderr_decoded = stderr.decode('utf-8')

        # get rid of old generation stuff
        os.remove("scene.py")
        os.rmdir("media")

        return json.dumps({
            "stdout": self.conv.convert(stdout_decoded),
            "stderr": self.conv.convert(stderr_decoded)
        })
        
    def _arun(self, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Generate a video segment asynchronously."""
        raise NotImplementedError("TextGenerate does not support async")
    
# INCLUDE PROMPTING STYLE IN TEACHING EFFECTIVENESS REPORT. LACK OF ATTENTION CAN BE BECAUSE OF SHITTY VISUALS
    
tool = [VideoGenerate()]
tool_oai = [format_tool_to_openai_tool(tool)]

code_creation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an agent whose job it is to write a python file which can be run using manim -pql scene.py ClassName to generate the animation or visual that the user requests."),
        ("user", "{desired_animation}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
code_creation_chain = (
    {
        "desired_animation": itemgetter("desired_animation"),
        "agent_scratchpad": itemgetter("intermediate_steps") | RunnableLambda(format_to_openai_tool_messages),
    }
    | code_creation_prompt
    | gpt_4_llm.bind(tools=tool_oai)
    | OpenAIToolsAgentOutputParser()
)

agent = AgentExecutor(agent=code_creation_chain, tools=tool, debug=True)

# 1. get basic video working (animations, equations, text)
# 2. incorporate visuals
# 3. incorporate plugins

code_output = agent.invoke({
    "desired_animation": "Create a visual of a spring oscillating, moving according to Hooke's Law. The spring should be attached to a wall on one end and a mass on the other. The mass should be displaced from its equilibrium position and then released. The mass should oscillate back and forth around the equilibrium position. The spring should be shown to stretch and compress as the mass moves. The mass should be shown to move with simple harmonic motion. The spring should be shown to obey Hooke's Law. The mass should be shown to obey Newton's Second Law. The mass should be shown to have kinetic energy and potential energy. The spring should be shown to have elastic potential energy. The mass should be shown to have maximum kinetic energy at the equilibrium position and maximum potential energy at the maximum displacement from the equilibrium position. The spring should be shown to have maximum elastic potential energy at the maximum displacement from the equilibrium position and maximum kinetic energy at the equilibrium position.",
})

AttributeError: 'list' object has no attribute 'args_schema'